In [24]:
import pandas as pd
import numpy as np
import sklearn
from urllib.request import urlopen
from json import loads

In [73]:
twenty_eleven_df = pd.read_csv('schools_crdc_ap_exams_2011.csv', low_memory=False)
twenty_thirteen_df = pd.read_csv('schools_crdc_ap_exams_2013.csv', low_memory=False)
twenty_fifteen_df = pd.read_csv('schools_crdc_ap_exams_2015.csv', low_memory=False)

# 2017's data has a lot of missing values and negative values. We don't consider using it at this moment
# twenty_seventeen_df = pd.read_csv('schools_crdc_ap_exams_2017.csv', low_memory=False)

In [144]:
# Read Enrollment Data
twenty_eleven_enroll_df = pd.read_csv("schools_crdc_enrollment_k12_2011.csv", low_memory=False)
twenty_thirteen_enroll_df = pd.read_csv("schools_crdc_enrollment_k12_2013.csv", low_memory=False)
twenty_fifteen_enroll_df = pd.read_csv("schools_crdc_enrollment_k12_2015.csv", low_memory=False)



7873490


In [145]:
# weird_twenty_eleven_df = twenty_eleven_df[
#     ((twenty_eleven_df['students_AP_exam_oneormore'] > twenty_eleven_df['students_AP_pass_oneormore']) & (twenty_eleven_df['students_AP_pass_oneormore'] >= 0)) | 
#     ((twenty_eleven_df['students_AP_exam_all'] > twenty_eleven_df['students_AP_exam_oneormore']) & (twenty_eleven_df['students_AP_exam_oneormore'] >= 0)) |
#     ((twenty_eleven_df['students_AP_pass_all'] > twenty_eleven_df['students_AP_exam_all']) & (twenty_eleven_df['students_AP_exam_all'] >= 0))
                                                                                                                        
# ]
# number_schools = weird_twenty_eleven_df['crdc_id'].nunique()
# print(len(weird_twenty_eleven_df))
# print(number_schools)

In [159]:
# missing_id = twenty_eleven_df[twenty_eleven_df['ncessch'].isna()]
# print(len(missing_id) / 30)

# Function to process data that removes all NA rows or with negative values

def data_validifier(dataframe, state_id):
    # Getting rows with the required state_id
    state_df = dataframe[dataframe['fips'] == state_id];
    
    # Aggregate Race and Sex
    state_agg = state_df[(state_df['race'] == 99) & (state_df['sex'] == 99)]
    
    # Drop some columns we don't need at this moment
    state_df_dropcolumn = state_agg.drop(columns=['students_AP_exam_all', 'students_AP_pass_all', 'fips', 'lep', 'disability', 'leaid', 'crdc_id', 
                                                'students_AP_exam_none', 'students_AP_pass_none', 'students_AP_exam_oneormore', 'sex', 'race'])
    
    # Drop NA Values(It's fine to have NA values for exam_all or pass_all
    state_df_nona = state_df_dropcolumn.dropna()
    
    # Remove all rows with negative values
    state_df_final = state_df_nona[(state_df_nona['students_AP_pass_oneormore'] >= 0)]
    
    return state_df_final

# Get California's valid df
california_2011 = data_validifier(twenty_eleven_df, 6)
california_2013 = data_validifier(twenty_thirteen_df, 6)
california_2015 = data_validifier(twenty_fifteen_df, 6)

print(california_2013)


        year  students_AP_pass_oneormore       ncessch
147329  2013                       188.0  6.000130e+10
147749  2013                       257.0  6.000131e+10
148767  2013                         4.0  6.000161e+10
148768  2013                         4.0  6.000161e+10
148769  2013                       291.0  6.000161e+10
...      ...                         ...           ...
417598  2013                         4.0  6.425101e+10
417599  2013                       140.0  6.425101e+10
417657  2013                         4.0  6.425101e+10
417659  2013                       307.0  6.425101e+10
417839  2013                        18.0  6.425401e+10

[1287 rows x 3 columns]


In [189]:
# Function to process enrollment data
def enrollment_processor(dataframe, state_id):
    # Getting rows with the relevant state_id
    state_enroll_df = dataframe[dataframe['fips'] == state_id]
    
    # Drop preenrollment 
    state_enroll_df = state_enroll_df.drop(columns=['psenrollment_crdc', 'crdc_id', 'fips'])
    
    # We need proportion of females and each races
    # We don't care about disability or LEP here
    state_enroll_relevant = state_enroll_df[(state_enroll_df['disability'] == 99) & (state_enroll_df['lep'] == 99)]
    
    # We only want data with ncessch id
    state_enroll_final = state_enroll_relevant[~state_enroll_relevant['ncessch'].isna()]
    
    return state_enroll_final

california_enroll_2011 = enrollment_processor(twenty
california_enroll_2015 = enrollment_processor(twenty_fifteen_enroll_df, 6)

    
    

Empty DataFrame
Columns: [year, ncessch, leaid, race, sex, disability, lep, enrollment_crdc]
Index: []


In [131]:
# Concat Data for training purpose
# Planning to use 2011 and 2013 for training

train_data = pd.concat([california_2011, california_2013], ignore_index = True)

# Drop the year column 
# Reset Index for test data
# train_data = train_data.drop(columns=['year'])
# test_data = california_2015.drop(columns=['year']).reset_index(drop=True)


train_feature = train_data.drop(columns=['students_AP_pass_none', 'students_AP_exam_none', 'students_AP_exam_oneormore', 'students_AP_pass_oneormore'])
train_none_ap = train_data['students_AP_exam_none']
train_none_passing = train_data['students_AP_pass_none']
train_oneormore_ap = train_data['students_AP_exam_oneormore']
train_oneormore_passing = train_data['students_AP_pass_oneormore']

test_feature = test_data.drop(columns=['students_AP_pass_none', 'students_AP_exam_none', 'students_AP_exam_oneormore', 'students_AP_pass_oneormore'])
test_none_ap = test_data['students_AP_exam_none']
test_none_passing = test_data['students_AP_pass_none']
test_oneormore_ap = test_data['students_AP_exam_oneormore']
test_oneormore_passing = test_data['students_AP_pass_oneormore']



,sex,race,ncessch
0,1,1.0,6.000010e+10
1,1,3.0,6.000010e+10
2,1,99.0,6.000010e+10
3,2,1.0,6.000010e+10
4,2,99.0,6.000010e+10
...,...,...,...
95,2,1.0,6.000201e+10
96,2,4.0,6.000201e+10
97,2,99.0,6.000201e+10
98,99,1.0,6.000201e+10


In [134]:
# Standard Scaling our features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(train_feature)
X_test = scaler.transform(test_feature)


In [135]:
# Use RandomForestRegressor to fit the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor(random_state = 482)

# Function to fit and predict different metrics
def predictor():
    
    # Fit and predict students_AP_exam_none
    model.fit(X_train, train_none_ap)
    predict_none_ap = model.predict(X_test)
    
    # Fit and predict students_AP_pass_none    
    model.fit(X_train, train_none_passing)
    predict_none_passing = model.predict(X_test)
    
    # Fit and predict students_AP_exam_oneormore
    model.fit(X_train, train_oneormore_ap)
    predict_oneormore_ap = model.predict(X_test)
    
    # Fit and predict students_AP_pass_oneormore
    model.fit(X_train, train_oneormore_passing)
    predict_oneormore_passing = model.predict(X_test)
    
    # Calculate the MSE metrics for each
    none_ap_mse = mean_squared_error(predict_none_ap, test_none_ap)
    none_passing_mse = mean_squared_error(predict_none_passing, test_none_passing)
    oneormore_ap_mse = mean_squared_error(predict_oneormore_ap, test_oneormore_ap)
    oneormore_passing_mse = mean_squared_error(predict_oneormore_passing, test_oneormore_passing)
    
    return none_ap_mse, none_passing_mse, oneormore_ap_mse, oneormore_passing_mse

none_ap_mse, none_passing_mse, oneormore_ap_mse, oneormore_passing_mse = predictor()
print("MSE for Students Taking No AP exams in California in 2015: ", none_ap_mse)
print("MSE for Students Passing No AP exams in California in 2015: ", none_passing_mse)
print("MSE for Students Taking One or More AP exams in California in 2015: ", oneormore_ap_mse)
print("MSE for Students Passing One or More AP exams in California in 2015: ", oneormore_passing_mse)
    
    

MSE for Students Taking No AP exams in California in 2015:  1696.0434014657346
MSE for Students Passing No AP exams in California in 2015:  1816.2829910415514
MSE for Students Taking One or More AP exams in California in 2015:  12083.408802793369
MSE for Students Passing One or More AP exams in California in 2015:  5679.750827187407
